In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import os
from IPython.core.debugger import set_trace


# Read text

In [2]:
def cleantxt(fname):
    rawname = fname+'_raw'
    if fname[-3:]=='txt':
        rawname = fname[:-4]+'_raw.txt'
    
    with open(fname, 'r') as f, open(rawname, 'a') as rawf:
        content = f.readlines()
        for line in f:
            rawf.write(line)
            
    # remove whitespace at the end. keep the tab \t to mark level of question    
    content = [x.strip('\n') for x in content] 
   
    for line in content:
        if line[0]==' ': # transform white spaces into tab which is used to count levels
            line = line.replace('  ','\t') 
        if line.find('？')>0: # deal with chinese characters
            line = line.replace('？','?')
        if line.find('。')>0:
            line = line.replace('。','.')
        with open(fname, 'w') as wf: # rewrite the original text name so nothing weird in the processed textfile.
            wf.write(line+'\n')
    return content,cleanname
        

# Text to quesnet
For paper notes, the first line will be the title of this paper, which should be linked to the paper data structure.




In [3]:
def getedge(head):
    # get edge
    if head.find('...')>0 and head.find('_')>0: # edge of a group head
        ed = head[head.find('_')+1:head.find('...')]
    else: # edge of single node
        ed = head[head.find('_')+1:head.rfind('_')]
    #unify the edge label names
    if ed == 'a' or ed.lower()=='answer':
        ed = 'answer' # rename the txt shorthand
    elif ed == 'specify':
        ed = 'specification'
    # get speaker
    if head.find('@')>0: # speaker other than this paper -- usually me
        speaker = head[head.find('@')+1:]
    else:
        speaker = ''
    return ed,speaker
def node2label(node):
    if node.find('"')>0:
        label = node[:node.find('"')]
        content = node[node.find('"'):]
    else:
        label = node
        content=""
    return label,content

In [4]:
def txt2quesnet(outdir,combrepQ = True,fin='',nodedist={},alledge=[]):
    for key in ['label','type','id','content','speaker','level']:
        nodedist[key]=[]
    with open(fin,'r') as f:
        content = f.readlines()
    
    # create a paper entity
    thisp = {'id':'Ep%d'%np.random.randint(100000)} #E=entity,p=paper
    #FUTURE: may use other ways to record the paper info
    fname = fin[fin.rfind('/')+1:] # get rid of the file directory 
    print(fname)
    thisp['label']=fname.strip('.txt')
    for ks,s in enumerate(fname):
        if s.isdigit():
            thisp['author'] = fname[:ks]
            break
    thisp['year']=fname[ks:ks+4]
    # if the first line is the paper name
    lstart=0 # which line to start reading
    if content[0]=='#':
        thisp['title']=content[0][3:]
        lstart=1
        
    # create multiple question threads from each root
    maxlevel = 20 # maximum 20 levels down the tree
    allngrps=[]
    for line in content[lstart:]:
        if len(line.strip())<1: # skip empty lines
            continue
        elif line[:2]=='##':#ending sign
            break
        else: # a valid new line
            line = line.rstrip()
            # new...
            if line[0].isalpha(): # a root node doesn't start with tab
                    rootlv=0
                    rootlist = [[] for k in range(maxlevel)]# restart a new root list
                    node = line
                    edge = ''
                    spkr = ''
                    head = ''
            else:
                head = line.split(' ')[0]
                if len(line.split(' '))==1:
                    if line[-3:]=='...': #group head
                        nodegrp = [] # for reading parallel nodes
                        hdedge,hdspkr = getedge(head)                    
                        continue
                    elif line.find('end_')>0:
                        allngrps.append(nodegrp)
                        continue
                    else:
                        print(line)
                        print("can't recognize your grammar...")  
                        continue
                else:
                    node = " ".join(line.split(' ')[1:])
                    if head.find('...')<0: # a normal edge; otherwise a member of edge group, edge is given
                        edge,spkr = getedge(head)
                    else:
                        edge,spkr = hdedge,hdspkr
        # Construct the node dictionary, get nid   
        level = int((len(line) - len(line.strip('\t')))/len('\t')) # an absolute level
        label,content=node2label(node)
        
        if combrepQ and label in nodedist['label']:
            nid = nodedist['id'][nodedist['label'].index(label)]
        else:
            if edge == 'answer' or edge =='hypothesis':
                nodedist['type'].append('Answer') 
                nid = 'a%d'%np.random.randint(100000)
            else:
                nid = 'q%d'%np.random.randint(100000)
                if level>0:
                    nodedist['type'].append('Question')    
                else:
                    nodedist['type'].append('Root question')
            nodedist['label'].append(label)
            nodedist['id'].append(nid)
            nodedist['content'].append(content)
            nodedist['level'].append(level)
            #nodedist['reference'].append(reference)
            if len(spkr)==0:
                spkr = thisp['id'] # if not specified, default speaker is this paper
            nodedist['speaker'].append(spkr)
        
        if head.find('...')>0:
            nodegrp.append(nid)       

        
        # Construct the edge dictionary    
        rootlist[level]=nid             
        if level>0:
            if len(rootlist[level-1])==0:
                print(node)
                set_trace()
            edgedict = {'start':rootlist[level-1],'end':nid,'label':edge,'id':'l%d'%np.random.randint(100000)}
            if rootlist[level-1] == nid:
                print(level)
                print(line)
                set_trace()
            alledge.append(edgedict)   

            
    nodedf = pd.DataFrame(nodedist)
    edgedf = pd.DataFrame(alledge) 
    #os.makedirs(filedir)
 
    pickle.dump([nodedf,edgedf,allngrps],open(outdir+thisp['label']+'_qnet.p','bw'))
    print('generated database in:\n'+outdir+thisp['label']+'_qnet.p')
    return thisp # output the paper entity. TODO: other entities mentioned in the nodes should also be added, so that return a full database

In [5]:
"""fname='../txtnotes/papernotes/Dezza2017_Learning'
outdir='../database/curiosity/compcogsci/'
Epaper=txt2quesnet(outdir,False,fname) """

"fname='../txtnotes/papernotes/Dezza2017_Learning'\noutdir='../database/curiosity/compcogsci/'\nEpaper=txt2quesnet(outdir,False,fname) "

In [6]:
fname='../database/curiosity/devpsych/nodeQs.txt'
outdir='../database/curiosity/devpsych/'
Epaper=txt2quesnet(outdir,False,fname) 

nodeQs.txt
generated database in:
../database/curiosity/devpsych/nodeQs_qnet.p


In [7]:
[nodedf,edgedf,allngrps] = pickle.load(open('../database/curiosity/compcogsci/'+fname[fname.rfind('/'):]+'_qnet.p','rb'))




FileNotFoundError: [Errno 2] No such file or directory: '../database/curiosity/compcogsci//nodeQs.txt_qnet.p'

next step: use "pickle to networkx" notebook to do the visualization

# Graph simplification

In [ ]:
def simpg(nodedf,edgedf,level=1):
    newndf = nodedf.loc[nodedf['level']<=level]
    newedf = []
    for row in edgedf.itertuples():
        
        if (nodedf.loc[nodedf['id']==row.start]['level'].values[0])<=(level-1):
            newedf.append(row)
    newedf = pd.DataFrame(newedf)
    return newndf,newedf

In [ ]:
[nodedf_l3,edgedf_l3] = simpg(nodedf,edgedf,level=3)
[nodedf_l2,edgedf_l2] = simpg(nodedf,edgedf,level=2)

pickle.dump([nodedf_l2,edgedf_l2,nodedf_l3,edgedf_l3,allngrps],open('../database/curiosity/compcogsci/Rich2019_the_qnet_simp.p','wb'))
